In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import joblib
from google.colab import files

import warnings
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, LabelEncoder, RobustScaler
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
from sklearn.metrics import make_scorer, f1_score
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import classification_report

# To ignore all warnings
warnings.filterwarnings("ignore")

The original dataset was taken from the following GitHub repository:
https://github.com/Byte7/Solar-Flares-Prediction-RHESSI-Mission/blob/master/data/hessi.solar.flare.2002to2016.csv

In [ ]:
df=pd.read_csv("/content/hessi.solar.flare.2002to2016.csv")

In [ ]:
df

,flare,start.date,start.time,peak,end,duration.s,peak.c/s,total.counts,energy.kev,x.pos.asec,y.pos.asec,radial,active.region.ar,flag.1,flag.2,flag.3,flag.4,flag.5
0,2021213,2002-02-12,21:29:56,21:33:38,21:41:48,712,136,167304,12-25,592,-358,692,0,A1,P1,NaN,NaN,NaN
1,2021228,2002-02-12,21:44:08,21:45:06,21:48:56,288,7,9504,6-12,604,-341,694,9811,A1,P1,PE,Q1,NaN
2,2021332,2002-02-13,00:53:24,00:54:54,00:57:00,216,15,11448,6-12,-310,375,487,9825,A1,P1,NaN,NaN,NaN
3,2021308,2002-02-13,04:22:52,04:23:50,04:26:56,244,20,17400,12-25,-277,378,469,9822,A1,P1,NaN,NaN,NaN
4,2021310,2002-02-13,07:03:52,07:05:14,07:07:48,236,336,313392,25-50,-272,390,476,9825,A1,GS,P1,PE,Q2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113937,16122603,2016-12-26,22:09:20,22:17:02,22:17:24,484,152,75328,3-6,0,0,0,0,A0,GE,NS,PE,Q2
113938,16122604,2016-12-26,23:25:32,23:28:18,23:43:48,1096,304,142048,3-6,0,0,0,0,A0,DR,NS,PE,Q2
113939,16122701,2016-12-27,08:33:08,08:39:06,08:39:40,392,80,32344,3-6,0,0,0,0,A0,DF,NS,PE,Q3 SE
113940,16122702,2016-12-27,12:21:24,12:21:30,12:30:04,520,24,17336,6-12,326,326,461,0,A0,DF,DR,P1,PE Q3


Removing records with certain flags: NS, SD, SS, SE, PS, Q12, Q11, Q10, Q9, Q8, Q7, Q6, Q5, Q4

In [ ]:
'''Filtering data based on the energy range 3-6 KeV, flags and radial values. Filtering possible values
    which may be wrong or without any solar event '''

def filter_data(DATA,lwrong):
    ## filter 1: 3-6 kev
    lenght1 = len(DATA)
    DATA = DATA[DATA['energy.kev']!='3-6']
    lenght2 = len(DATA)
    if lenght1!=lenght2:
        print('filtering 3-6 kev energy range',lenght1,lenght2)


    # filter possible wrong values or without solar event
    for icod in lwrong:
        ## filter
        lenght1 = len(DATA)
        DATA = DATA[DATA['flag.1']!=icod]
        lenght2 = len(DATA)
        if lenght1!=lenght2:
            print('filtering %s'%icod,lenght1,lenght2)

        lenght1 = len(DATA)
        DATA = DATA[DATA['flag.2']!=icod]
        lenght2 = len(DATA)
        if lenght1!=lenght2:
            print('filtering %s'%icod,lenght1,lenght2)

        lenght1 = len(DATA)
        DATA = DATA[DATA['flag.3']!=icod]
        lenght2 = len(DATA)
        if lenght1!=lenght2:
            print('filtering %s'%icod,lenght1,lenght2)

        lenght1 = len(DATA)
        DATA = DATA[DATA['flag.4']!=icod]
        lenght2 = len(DATA)
        if lenght1!=lenght2:
            print('filtering %s'%icod,lenght1,lenght2)

        lenght1 = len(DATA)
        values = DATA['flag.5'].values
        lfilter = [i for i in list(values[pd.notnull(values)]) if icod in i]
        DATA = DATA[~DATA['flag.5'].isin(lfilter)]
        lenght2 = len(DATA)
        if lenght1!=lenght2:
            print('filtering %s'%icod,lenght1,lenght2)

    return DATA

In [ ]:

# filtering basic
lwrong = ['NS','SD','SS','SE','PS','Q12','Q11','Q10','Q9','Q8','Q7','Q6','Q5','Q4']
filtered_data = filter_data(df,lwrong)

filtering 3-6 kev energy range 113942 107606
filtering SD 107606 107570
filtering SD 107570 107476
filtering SS 107476 107469
filtering SS 107469 107441
filtering SE 107441 106999
filtering SE 106999 106035
filtering Q5 106035 105985
filtering Q4 105985 104718


In [ ]:
filtered_data

,flare,start.date,start.time,peak,end,duration.s,peak.c/s,total.counts,energy.kev,x.pos.asec,y.pos.asec,radial,active.region.ar,flag.1,flag.2,flag.3,flag.4,flag.5
0,2021213,2002-02-12,21:29:56,21:33:38,21:41:48,712,136,167304,12-25,592,-358,692,0,A1,P1,NaN,NaN,NaN
1,2021228,2002-02-12,21:44:08,21:45:06,21:48:56,288,7,9504,6-12,604,-341,694,9811,A1,P1,PE,Q1,NaN
2,2021332,2002-02-13,00:53:24,00:54:54,00:57:00,216,15,11448,6-12,-310,375,487,9825,A1,P1,NaN,NaN,NaN
3,2021308,2002-02-13,04:22:52,04:23:50,04:26:56,244,20,17400,12-25,-277,378,469,9822,A1,P1,NaN,NaN,NaN
4,2021310,2002-02-13,07:03:52,07:05:14,07:07:48,236,336,313392,25-50,-272,390,476,9825,A1,GS,P1,PE,Q2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113930,16121301,2016-12-13,15:40:12,15:41:14,15:43:36,204,24,8096,6-12,-336,260,425,0,A0,DR,EE,P1,PE Q3
113931,16121501,2016-12-15,03:26:36,03:30:06,03:38:32,716,136,76016,12-25,929,307,979,2617,A0,DR,P1,PE,Q2
113934,16122102,2016-12-21,16:16:40,16:16:42,16:16:48,8,24,368,6-12,914,-84,918,0,A0,DF,P1,PE,Q2
113940,16122702,2016-12-27,12:21:24,12:21:30,12:30:04,520,24,17336,6-12,326,326,461,0,A0,DF,DR,P1,PE Q3


In [ ]:
#Splitting date to Month, Day and Year
# process date / time columns
def parse_date(sdatex,stimex):
    datex = datetime.strptime(sdatex, '%Y-%m-%d')
    timex = datetime.strptime(stimex, '%H:%M:%S')
    return datetime(datex.year,datex.month,datex.day,timex.hour,timex.minute,timex.second)

# Adding year, month, day, start date, peak date, end date and dropping earlier columns
filtered_data['dt.start'] = filtered_data[['start.date','start.time']].apply(lambda x: parse_date(x[0],x[1]), axis=1)
filtered_data['dt.peak'] = filtered_data[['start.date','peak']].apply(lambda x: parse_date(x[0],x[1]), axis=1)
filtered_data['dt.end'] = filtered_data[['start.date','end']].apply(lambda x: parse_date(x[0],x[1]), axis=1)

# clean columns
filtered_data.drop(['start.date','start.time','peak','end'], axis=1, inplace=True)

# add new columns
filtered_data['year'] = filtered_data['dt.start'].apply(lambda col: col.year)
filtered_data['month'] = filtered_data['dt.start'].apply(lambda col: col.month)
filtered_data['day'] = filtered_data['dt.start'].apply(lambda col: col.day)
filtered_data.drop(['dt.start','dt.peak','dt.end'], axis=1, inplace=True)

In [ ]:
filtered_data

,flare,duration.s,peak.c/s,total.counts,energy.kev,x.pos.asec,y.pos.asec,radial,active.region.ar,flag.1,flag.2,flag.3,flag.4,flag.5,year,month,day
0,2021213,712,136,167304,12-25,592,-358,692,0,A1,P1,NaN,NaN,NaN,2002,2,12
1,2021228,288,7,9504,6-12,604,-341,694,9811,A1,P1,PE,Q1,NaN,2002,2,12
2,2021332,216,15,11448,6-12,-310,375,487,9825,A1,P1,NaN,NaN,NaN,2002,2,13
3,2021308,244,20,17400,12-25,-277,378,469,9822,A1,P1,NaN,NaN,NaN,2002,2,13
4,2021310,236,336,313392,25-50,-272,390,476,9825,A1,GS,P1,PE,Q2,2002,2,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113930,16121301,204,24,8096,6-12,-336,260,425,0,A0,DR,EE,P1,PE Q3,2016,12,13
113931,16121501,716,136,76016,12-25,929,307,979,2617,A0,DR,P1,PE,Q2,2016,12,15
113934,16122102,8,24,368,6-12,914,-84,918,0,A0,DF,P1,PE,Q2,2016,12,21
113940,16122702,520,24,17336,6-12,326,326,461,0,A0,DF,DR,P1,PE Q3,2016,12,27


In [ ]:
#Drop flare, flag.1, flag.2, flag.3, flag.4, flag.5

# List of columns to be dropped
columns_to_drop = ['flare', 'flag.1', 'flag.2', 'flag.3', 'flag.4', 'flag.5','active.region.ar']

# Drop the specified columns
df_dropped = filtered_data.drop(columns=columns_to_drop)

# Display the DataFrame after dropping columns
print("\nDataFrame after dropping columns:\n\n\n")
df_dropped


DataFrame after dropping columns:





,duration.s,peak.c/s,total.counts,energy.kev,x.pos.asec,y.pos.asec,radial,year,month,day
0,712,136,167304,12-25,592,-358,692,2002,2,12
1,288,7,9504,6-12,604,-341,694,2002,2,12
2,216,15,11448,6-12,-310,375,487,2002,2,13
3,244,20,17400,12-25,-277,378,469,2002,2,13
4,236,336,313392,25-50,-272,390,476,2002,2,13
...,...,...,...,...,...,...,...,...,...,...
113930,204,24,8096,6-12,-336,260,425,2016,12,13
113931,716,136,76016,12-25,929,307,979,2016,12,15
113934,8,24,368,6-12,914,-84,918,2016,12,21
113940,520,24,17336,6-12,326,326,461,2016,12,27


In [ ]:
'''columns_to_drop = ['start.time','peak','end','flare', 'flag.1', 'flag.2', 'flag.3', 'flag.4', 'flag.5','active.region.ar']

# Drop the specified columns
df_dropped1 = filtered_data.drop(columns=columns_to_drop)

# Display the DataFrame after dropping columns
print("\nDataFrame after dropping columns:\n\n\n")
df_dropped1'''


DataFrame after dropping columns:





,start.date,duration.s,peak.c/s,total.counts,energy.kev,x.pos.asec,y.pos.asec,radial
0,2002-02-12,712,136,167304,12-25,592,-358,692
1,2002-02-12,288,7,9504,6-12,604,-341,694
2,2002-02-13,216,15,11448,6-12,-310,375,487
3,2002-02-13,244,20,17400,12-25,-277,378,469
4,2002-02-13,236,336,313392,25-50,-272,390,476
...,...,...,...,...,...,...,...,...
113930,2016-12-13,204,24,8096,6-12,-336,260,425
113931,2016-12-15,716,136,76016,12-25,929,307,979
113934,2016-12-21,8,24,368,6-12,914,-84,918
113940,2016-12-27,520,24,17336,6-12,326,326,461


In [ ]:
#Number of energy band classes after data cleaning
c=df_dropped.groupby("energy.kev")["energy.kev"].count()
print(c)

energy.kev
100-300         155
12-25         18417
25-50          1806
300-800           8
50-100          359
6-12          83970
7000-20000        1
800-7000          2
Name: energy.kev, dtype: int64


**Data Split: Training, Test and Unseen**

In [ ]:
rare_categories = df_dropped['energy.kev'].value_counts()[df_dropped['energy.kev'].value_counts() == 1].index

# Create a DataFrame conaining only rows of rare categories
duplicate_df = df_dropped[df_dropped['energy.kev'].isin(rare_categories)].copy()

# Duplicate the records
df_dropped = pd.concat([df_dropped, duplicate_df], ignore_index=True)
y = df_dropped['energy.kev']

# Use StratifiedShuffleSplit to split the data
sss = StratifiedShuffleSplit(n_splits=2, test_size=0.25, random_state=42)

for train_index, test_index in sss.split(df_dropped, y):
    train_set = df_dropped.iloc[train_index]
    test_unseen_set = df_dropped.iloc[test_index]

# Further split the test_unseen_set into test_set and unseen_set
test_set, unseen_set = train_test_split(
    test_unseen_set,
    test_size=0.5,
    random_state=42,
    stratify=test_unseen_set['energy.kev']
)

# Display the shapes of the resulting sets
print("Shapes of the sets:")
print("Train set:", train_set.shape)
print("Test set:", test_set.shape)
print("Unseen set:", unseen_set.shape)

Shapes of the sets:
Train set: (78539, 10)
Test set: (13090, 10)
Unseen set: (13090, 10)


In [ ]:
train_set.to_csv('train_set.csv', index=False)
test_set.to_csv('test_set.csv', index=False)
unseen_set.to_csv('unseen_set.csv', index=False)

**Pipeline and Model**

In [ ]:
df1=pd.read_csv('/content/train_set.csv')
df2=pd.read_csv('/content/test_set.csv')

In [ ]:
X_train=df1.drop('energy.kev', axis=1)
y_train=df1['energy.kev']
X_test=df2.drop('energy.kev', axis=1)
y_test=df2['energy.kev']
y_train.unique()

array(['6-12', '12-25', '25-50', '100-300', '50-100', '7000-20000',
       '800-7000', '300-800'], dtype=object)

In [ ]:
#Encoding the target Column
y_train=np.ravel(y_train)
y_test=np.ravel(y_test)

encoder = LabelEncoder()
target_categories = ['6-12', '12-25', '25-50', '100-300', '50-100', '300-800','800-7000','7000-20000']
encoder.fit(target_categories)
y_train=encoder.transform(y_train)
y_test=encoder.transform(y_test)

Creating Pipeline after Hyperparameter Tuning

In [ ]:
pipeline = Pipeline([
     #('preprocessor', preprocessor),
    ('scaler', RobustScaler()),
    ('classifier', XGBClassifier(max_depth=7, n_estimators=200,learning_rate=0.1,colsample_bytree=1.0,subsample=0.8))
])

Checking Overfitting With Stratified K fold

In [ ]:
# Specify the scoring metric (F1 score)
scorer = make_scorer(f1_score, average='weighted')  # 'weighted' for multi-class classification

# Specify the number of folds for Stratified K-Fold cross-validation
n_folds = 5

# Instantiate StratifiedKFold
stratified_kfold = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)

# Perform cross-validation and calculate F1 scores
f1_scores = cross_val_score(pipeline, X_train, y_train, cv=stratified_kfold, scoring=scorer)

# Print the F1 scores for each fold
for fold, f1_score_value in enumerate(f1_scores, 1):
    print(f"Fold {fold}: F1 Score = {f1_score_value:.4f}")

# Print the average F1 score across all folds
average_f1_score = f1_scores.mean()
print(f"\nAverage F1 Score across all folds: {average_f1_score:.4f}")

Fold 1: F1 Score = 0.8603
Fold 2: F1 Score = 0.8589
Fold 3: F1 Score = 0.8612
Fold 4: F1 Score = 0.8576
Fold 5: F1 Score = 0.8624

Average F1 Score across all folds: 0.8601


In [ ]:
pipeline.fit(X_train, y_train)


# Predict on the test set
y_pred = pipeline.predict(X_test)

# Evaluate the performance
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.11      0.05      0.07        19
           1       0.70      0.59      0.64      2302
           2       0.35      0.04      0.06       226
           3       0.00      0.00      0.00         1
           4       0.33      0.04      0.08        45
           5       0.91      0.96      0.93     10497

    accuracy                           0.88     13090
   macro avg       0.40      0.28      0.30     13090
weighted avg       0.86      0.88      0.86     13090



Download Encoder and Model

In [ ]:
joblib.dump(pipeline, 'trained_pipeline.joblib')
joblib.dump(encoder, 'label_encoder.joblib')

['label_encoder.joblib']

**Implement on Unseen data**

In [ ]:
df3=pd.read_csv('/content/unseen_set.csv')
x=df3.drop('energy.kev', axis=1)
y=df3['energy.kev']

In [ ]:
loaded_pipeline = joblib.load('/content/trained_pipeline.joblib')
loaded_encoder=joblib.load('/content/label_encoder.joblib')

In [ ]:
y=loaded_encoder.transform(y)

In [ ]:
predictions = loaded_pipeline.predict(x)

In [ ]:
print(classification_report(y, predictions))

              precision    recall  f1-score   support

           0       0.75      0.30      0.43        20
           1       0.68      0.56      0.61      2302
           2       0.33      0.04      0.06       226
           3       0.00      0.00      0.00         1
           4       0.43      0.07      0.12        45
           5       0.90      0.96      0.93     10496

    accuracy                           0.87     13090
   macro avg       0.52      0.32      0.36     13090
weighted avg       0.85      0.87      0.86     13090

